In [16]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import time
import pymongo
from flask import Flask, render_template
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager

In [25]:
# !which chromedriver

chromedriver not found


In [14]:
# I couldn't locate my chromedriver, so I worked around it with a solution from class
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [/Users/kimcrine/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


# NASA Mars News

In [26]:
# URL to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# HTML Object
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

# Collect the latest element with news title and paragraph
news_title = soup.find_all('div', class_='content_title')[0].text
news_p = soup.find_all('div', class_='article_teaser_body')[0].text

# Display scrapped data 
print(news_title)
print(news_p)

Mars Now
The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from functional to decorative.


# JPL Mars Space Images - Featured Image

In [29]:
# URL to be scraped
featured_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(featured_image_url)

# HTML Object 
html_image = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html_image, "html.parser")

# Collect background-image url from style tag 
image_url  = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]

# Website Url 
main_url = "https://www.jpl.nasa.gov"

# Join website url with scrapped route
image_url = main_url + image_url

# Display full link to featured image
print(image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA22949-1920x1200.jpg
